In [16]:
%reload_ext snakeviz
%run 'MC.py'

In [30]:
lattice(8)[(1,0,1)].index

9

In [2]:
group(3).elements

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]

In [29]:
sys = hypertree_model(4,2,(1.,0.1))
print(list(range(sys.Nsite)))
for step in range(10):
    sys.MCstep()
    print(sys.config)#, sys.energy())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]


In [38]:
def test():
    sys = hypertree_model(128,2,(1.,0.1))
    #energies = []
    for step in range(32):
        sys.MCstep()
        #energies.append(sys.energy())
%snakeviz test()

 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmpbklz1_ur'. 


In [17]:
def test():
    sys = hypertree_model(128,2,(1.,0.1))
    #energies = []
    for step in range(32):
        sys.MCstep()
        #energies.append(sys.energy())
%snakeviz test()

 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmp3m3zubth'. 


In [10]:
%run 'MC.py'
import MC
class model(object):
    def __init__(self):
        MC.core.init()
    def config(self):
        return MC.core.config-1
    def MCstep(self):
        MC.core.MCstep()
class hypertree_model(model):
    def __init__(self, L=0, n=0, Ks=(1.,1.), branches=2):
        # setup the lattice
        latt = lattice(L, branches=branches)
        self.latt = latt
        # setup the permutation group
        grp = group(n)
        self.grp = grp
        dof = grp.order
        # passing data to FORTRAN MC core
        MC.core.dof = dof
        MC.core.Nsite = latt.Nsite
        MC.core.NA, MC.core.NB, MC.core.NC = latt.NA, latt.NB, latt.NC
        # initialize configuration
        MC.core.config = numpy.array([1]*latt.Nsite)
        # chi is adjusted by dof
        MC.core.chi = numpy.array(grp.chi)-dof
        # build Hamiltonian
        H = []
        for a in latt:
            i = a.index # take site a index
            bs = a.IR # get its IR sites
            for b, K in zip(bs, Ks):
                j = b.index # take site b index
                H.append((i,j,K))
                H.append((j,i,K))
        ilst, jlst, Klst = tuple(zip(*sorted(H)))
        MC.core.irng = self.make_irng(ilst)+1
        MC.core.jlst = numpy.array(jlst)+1
        MC.core.Klst = numpy.array(Klst)
        model.__init__(self)
    # convert ilst to irng
    def make_irng(self, ilst):
        imax = 0
        irng = [0]
        for k, i in zip(range(len(ilst)),ilst):
            while i > imax:
                irng.append(k)
                imax += 1
        irng.append(k+1)
        return numpy.array(irng)

In [11]:
sys = hypertree_model(4,2,(1.,0.2))
sys.config()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [8]:
MC.core.MCstep()
MC.core.config

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [4]:
import MC
print(MC.__doc__)

This module 'MC' is auto-generated with f2py (version:2).
Functions:
Fortran 90/95 modules:
  core --- config,irng,jlst,dof,Klst,NA,NC,NB,chi,Nsite,init(),MCstep(),dump(),load().


In [8]:
MC.core.config[1]=1

In [8]:
MC.core.dump()

In [13]:
sys.MCstep()
sys.config()

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [4]:
numpy.array([1,2,3])

dtype('int64')